## This is the week 3 assignment for the capstone project

Install and import the necessary libraries

In [115]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [116]:
!pip install BeautifulSoup4
!pip install bs4
!pip install requests
!pip install lxml

In [117]:
import requests
from bs4 import BeautifulSoup
import lxml
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Get the table from the Wikipedia page using BeautifulSoup.

In [118]:
webpage = requests.get(url).text

In [119]:
soup = BeautifulSoup(webpage, 'html.parser')

In [120]:
table = soup.find('table',  class_='wikitable sortable')

In [121]:
col_pos =[]
col_bo =[]
col_neigh = []

Save the table in the new lists: pos, boro and neigh after cleaning up the raw string, so that trailing spaces are removed.
Remove the row if the borough is "Not assigned".
If the Neighborhood value is "Not assigned", set it as the corresponding Borough value.

In [122]:
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        
        col_pos.append(cells[0].find(text=True))
        col_bo.append(cells[1].find(text=True))
        col_neigh.append(cells[2].find(text=True))

pos =[]
boro =[]
neigh =[]
for i in range(0, len(col_pos)):
    tempboro = col_bo[i]
    tempnei  = col_neigh[i]
    if "Not assigned" not in tempboro:
        pos.append(col_pos[i].rstrip())
        boro.append(col_bo[i].rstrip())
        if "Not assigned" in tempnei:
            neigh.append(col_bo[i].rstrip())
        else:
            neigh.append(col_neigh[i].rstrip())

Save the table as a pandas dataframe and check its form

In [142]:
df = pd.DataFrame({"Postal Code":pos, "Borough":boro, "Neighborhood":neigh})
df.head

'M3A'

Check the number of rows

In [125]:
df.shape

(103, 3)

Read the geogspatial coordiantes from the link

In [152]:
geodat = pd.read_csv("http://cocl.us/Geospatial_data")

Check the format

In [157]:
geodat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Inner join the wikipedia table with the geospatial coordinates using 'Postal Code' as the key. The previous key from the Wikipedia table 'PostalCode' has been modified to 'Postal Code' in order to match the geospatial coordinates

In [154]:
df_new = pd.merge(df, geodat, on = 'Postal Code' , how = 'inner')

In [156]:
df_new

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
